# Unit Tests for the Global Verification Summary Module

## Component Being Tested
Subroutines in global_summary.F90 (but excluding the MPI communications).

## Code

In ACME's code repository. Currently on branch huiwanpnnl/atm/global_verif_smry.

Path: ACME/components/cam/src/physics/granular_tests/global_summary_reorg/

## Purpose

During an ACME-atmosphere model simulation, a number of global fields are monitored for unphysical values such as negative tracer concentrations or excessive moisture flux from the atmosphere to the Earth's surface. In the standard model, warning messages (when switched on) are printed to atm.log and acme.log from every chunk of the physics grid, which often leads to excessively large log files. The module in ACME/components/cam/src/physics/global_summary.F90 provides concise summaries of the unphysical values. The information printed to the log files include:
- the total number of grid cells ( on the entire globe) in which unphysical values occur,
- the worst violation, and 
- when in verbose mode, the locations (latitude, longitude, vertical leve, column and chunk indices) at which the worst values occur.

This set of unit tests checks whether the subroutines in global_summary.F90 work as intended.

## Tests

### Initialization
The initialization phase tests the following functionalities:
- registering fields for global summary,
- allocating memory for the arrays that store the chunk summary, domain (single MPI process) summary, and global summary.

### Testing the summary subroutines
The basic method used in the tests is to get two global summaries for the same field, once for values (or absolute values) greater than or equal to a threshold, and once for values (or absolute values) smaller than the same threshold, add the total counts of violation occurance from the two summaries, then check whether
- within each chunk, the sum of the two counts matches the total number of cells in the chunk,
- within each domain (MPI process), the sum of the two counts matches the total number of cells in the domain,
- for each field and each type of summary, the sum of the counts from all chunks match the total count in the domain.

### Testing the clipping
A clipping option is implemented as an option to reset the exceeding values to the threshold. Clipping is turned on during field registration. To test whether the clipping works as extended, we
- define a tracer called NEGCOLIDX_FIX, and initialize it with -1._r8*icol where icol is the column index, and turn on clipping.
- register two summaries, one for NEGCOLIDX_FIX <= -4.5 and the other for NEGCOLIDX_FIX > -4.0
- use the module to get global summeries, then check whether
  - the sum of the total counts matches the total number of cells in a chunk/domain
  - the minimum and maximum values match the thresholds.

# Results

All the tests described above are coded in a single Fortran subroutine. If the program runs successfully and gives correct results, the following line will be printed to stdout (or a log file):

"all results conformed with expected values"